In [1]:
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_openai import ChatOpenAI
from langchain_core.tools import BaseTool
import os
from dotenv import load_dotenv
load_dotenv()

class WebBrowserTool(BaseTool):
    name: str = "google_serper"
    description: str = "Tool untuk web browsing"

    def _run(self, input: str):
        search = GoogleSerperAPIWrapper(serper_api_key=os.getenv("SERPER_API_KEY"))
        results = search.results(input)

        return results

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("human", "{input}"),
        # Placeholders fill up a **list** of messages
        ("placeholder", "{agent_scratchpad}"),
    ]
)

model = ChatOpenAI(model='gpt-4o-mini', api_key=os.getenv("OPENAI_API_KEY"))
tools = [WebBrowserTool()]

agent = create_tool_calling_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, return_intermediate_steps=True)

In [48]:
prompt = """
Tugasmu adalah mencari metadata dari perusahaan {company}.
Kamu dapat menggunakan tools google_serper untuk mencari metadata dari internet.
Berikut ini adalah daftar metadata yang harus dicari!
    company_name: str = Field(
        ..., 
        description="Nama perusahaan berdasarkan badan hukumnya", 
        examples=["Indofood CBP Sukses Makmur Tbk"]
    )
    country: str = Field(
        ..., 
        description="Negara asal perusahaan, bukan negara tempat beroperasi", 
        examples=["Indonesia"]
    )
    region: str = Field(
        ..., 
        description="Region asal perusahaan", 
        examples=["South East Asia", "Europe"]
    )
    sector: str = Field(
        ..., 
        description="Sektor utama di mana perusahaan beroperasi", 
        examples=["Consumer Goods", "Financial Services"]
    )
    subsector: str = Field(
        ..., 
        description="Subsektor yang lebih spesifik dari sektor utama", 
        examples=["Food and Beverages", "Banking"]
    )
    industry: str = Field(
        ..., 
        description="Industri spesifik yang terkait dengan subsektor", 
        examples=["Packaged Foods", "Commercial Banks"]
    )
    subindustry: str = Field(
        ..., 
        description="Subindustri yang lebih rinci dari industri", 
        examples=["Instant Noodles", "Retail Banking"]
    )
    ticker: Optional[str] = Field(
        ..., 
        description="Kode saham perusahaan (jika tersedia)", 
        examples=["ICBP", "BBCA"]
    )
    website: Optional[str] = Field(
        None, 
        description="URL resmi dari website perusahaan", 
        examples=["https://www.indofood.com"]
    )

"""

formatted_prompt = prompt.format(company="Merck Tbk")
response = agent_executor.invoke({"input": formatted_prompt})

In [49]:
from pydantic import BaseModel, Field
from typing import Optional

class CompanyMetadata(BaseModel):
    company_name: str = Field(
        ..., 
        description="Nama perusahaan berdasarkan badan hukumnya", 
        examples=["Indofood CBP Sukses Makmur Tbk"]
    )
    country: str = Field(
        ..., 
        description="Negara asal perusahaan, bukan negara tempat beroperasi", 
        examples=["Indonesia"]
    )
    region: str = Field(
        ..., 
        description="Region asal perusahaan", 
        examples=["South East Asia", "Europe"]
    )
    sector: str = Field(
        ..., 
        description="Sektor utama di mana perusahaan beroperasi", 
        examples=["Consumer Goods", "Financial Services"]
    )
    subsector: str = Field(
        ..., 
        description="Subsektor yang lebih spesifik dari sektor utama", 
        examples=["Food and Beverages", "Banking"]
    )
    industry: str = Field(
        ..., 
        description="Industri spesifik yang terkait dengan subsektor", 
        examples=["Packaged Foods", "Commercial Banks"]
    )
    subindustry: str = Field(
        ..., 
        description="Subindustri yang lebih rinci dari industri", 
        examples=["Instant Noodles", "Retail Banking"]
    )
    ticker: Optional[str] = Field(
        ..., 
        description="Kode saham perusahaan (jika tersedia)", 
        examples=["ICBP", "BBCA"]
    )
    website: Optional[str] = Field(
        None, 
        description="URL resmi dari website perusahaan", 
        examples=["https://www.indofood.com"]
    )


In [50]:
sllm = model.with_structured_output(CompanyMetadata)
structured_response = sllm.invoke(response['output'])

In [51]:
structured_response.model_dump()

{'company_name': 'PT Merck Tbk',
 'country': 'Indonesia',
 'region': 'South East Asia',
 'sector': 'Healthcare',
 'subsector': 'Pharmaceuticals',
 'industry': 'Drug Manufacturers - Specialty & Generic',
 'subindustry': 'Prescription Medicines',
 'ticker': 'MERK',
 'website': 'https://www.merckgroup.com/id-en'}